In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.formula as sm

In [35]:
def DGP(mu,sigma,size,y0):
    ut = np.random.normal(mu,sigma,size)
    data = pd.DataFrame([])
    data['ut']= ut
    data.insert(0,'yt',np.nan)
    data.insert(1,'lag_yt',np.nan)
    data.iloc[0,1] = y0
    for i in range(0,len(data.index),1):
        if i >0:
            data.iloc[i,1]=data.iloc[i-1,0]
        data.iloc[i,0]=0.8*data.iloc[i,1]+data.iloc[i,2]
    return data

In [3]:
def bst(data):
    return np.random.choice(data,size=len(data))

In [72]:
def est(data):
    model = smf.ols('yt~lag_yt-1',data).fit()
    p = model.params[0]
    return p

In [84]:
def bootstrap_ci(
    data,
    statfunction, 
    studentized = True,
    n_bs_samples=1000,
    n_inner_samples=100,
    ptiles=(2.5, 97.5),
):
    var = np.array(np.empty(n_bs_samples))
    ts = np.array(np.zeros(n_bs_samples))

    # Outer sample
    if studentized == True:
        for i in range(n_bs_samples):
            sample = draw_bs_sample(data)
            theta  = statfunction(sample)
            var[i] = theta
            
            resample_stats = np.empty(n_inner_samples)
            for j in range(n_inner_samples):
                resample = draw_bs_sample(sample)
                resample_stats[j] = statfunction(resample)

        # Compute statistic of interest
        ts[i] = np.sum(resample_stats - theta) / np.std(resample_stats)

        s = np.std(thetas)
        ts = np.sort(ts)

        p1, p2 = np.percentile(s*ts, ptiles)
        ci_high = np.mean(data)-(s*p1)
        ci_low = np.mean(data)-(s*p2)
        return (ci_low, ci_high)
    else: 
        for i in range(n_bs_samples):
            sample = DGP(mu,sigma,30,y0)
            theta = statfunction(sample)
            var[i] = theta

        # Inner sample
        resample_stats = np.empty(n_inner_samples)
        for j in range(n_inner_samples):
            resample = DGP(mu,sigma,30,y0)
            resample_stats[j] = statfunction(resample)

        # Compute statistic of interest
        ts[i] = np.sum(resample_stats - theta) / np.std(resample_stats)

    s = np.std(var)
    ts = np.sort(ts)

    p1, p2 = np.percentile(s*ts, ptiles)
    ci_high = np.mean(data)-(s*p1)
    ci_low = np.mean(data)-(s*p2)
    return (ci_low, ci_high)

In [7]:
mu = 0
sigma = 0.36**(-1/2)
y0 = np.random.normal(0, sigma, 1)

In [49]:
data42=DGP(mu,sigma,30,y0)

In [50]:
data42.head()

,yt,lag_yt,ut
0,1.302170,0.478729,0.919186
1,-0.595522,1.302170,-1.637258
2,-4.415780,-0.595522,-3.939362
3,-2.912658,-4.415780,0.619966
4,-1.212275,-2.912658,1.117852


In [51]:
model = smf.ols('yt ~ lag_yt -1',data42).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                     yt   R-squared (uncentered):                   0.744
Model:                            OLS   Adj. R-squared (uncentered):              0.735
Method:                 Least Squares   F-statistic:                              84.34
Date:                Wed, 15 Jun 2022   Prob (F-statistic):                    4.39e-10
Time:                        21:11:31   Log-Likelihood:                         -58.479
No. Observations:                  30   AIC:                                      119.0
Df Residuals:                      29   BIC:                                      120.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [53]:
beta = model.params[0]

In [85]:
bootstrap_ci(
    [],
    statfunction = est,
    studentized = False,
    n_bs_samples=100,
    n_inner_samples=10,
    ptiles=(2.5, 97.5),
)

/Users/ceciliah/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ceciliah/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(nan, nan)

Q3

In [88]:
earnings=pd.read_csv('earnings-data.csv')
earnings.head()

FileNotFoundError: [Errno 2] No such file or directory: 'earnings-data.csv'